## Setup

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')))

In [ ]:
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!
import model_setup

In [5]:
setup = model_setup.ModelSetup()

✅ Openrouter API Key exists and begins with: sk-or-v1
✅ Google API Key exists and begins with: AIzaSyDO


In [6]:
setup._check_const()

OPENAI_MODEL: openai/gpt-4o-mini
DEEPSEEK_MODEL: deepseek/deepseek-chat-v3-0324
GEMINI_MODEL: gemini-2.0-flash
OPENROUTER_URL: https://openrouter.ai/api/v1


In [8]:
system_message = "You are a helpful assistant"

client = OpenAI(
    api_key=setup.openrouter_api_key, 
    base_url=setup.OPENROUTER_URL
)

## Chatbot

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = client.chat.completions.create(model=setup.OPENAI_MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hello there'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'Hello there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'metadata': None, 'content': 'Hello there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}, {'role': 'user', 'content': 'I want to buy a tie'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'Hello there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'I want to buy a tie', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Great!

## Custom Systemprompt Chatbot

In [11]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = client.chat.completions.create(model=setup.OPENAI_MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [13]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [14]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [17]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = client.chat.completions.create(model=setup.OPENAI_MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [18]:
gr.ChatInterface(fn=chat, type="messages").launch()                     

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
